# INSERTION DE NOUVELLES DONNEES

###### 

### Préparation des données

In [16]:
import os
import pandas as pd
import psycopg2
import csv
import reprlib
import numpy
from translate import Translator
from googletrans import Translator

In [ ]:
# connexion à la bdd collection
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection4"
)
cur = conn.cursor()
cur.close
conn.close()

- nettoyage des données
- sauvegarder les données nettoyées dans un fichier csv
- insertion nouveau produit:
    - si origine n'existe pas:
        - ajouter dans table origine
    - si producteur n'existe pas:
        - ajouter dans table producteur

###### 

- nettoyage des nouvelles données

In [ ]:
import pandas as pd

In [ ]:
csv_file = 'rhumfullinfo_wikirum.csv'
new_data = pd.read_csv(csv_file)

new_data.head()

In [ ]:
new_data.isnull().sum()

In [ ]:
index_with_nan = new_data.index[new_data.isnull().any(axis=1)]
new_data.drop(index_with_nan,0, inplace=True)

In [ ]:
new_data.columns = ['nom', 'producteur', 'origine', 'volume', 'degre']
new_data['producteur'] = new_data['producteur'].str.replace('*','', regex=True)
new_data['nom'] = new_data['nom'].str.replace('\'','', regex=True)
new_data['producteur'] = new_data['producteur'].str.replace('\'','', regex=True)
new_data['degre'] = new_data['degre'].astype(float)
new_data['volume'] = new_data['volume'].astype(int)
new_data['volume'] = new_data['volume'] / 10

In [ ]:
new_data

In [ ]:
new_data.shape

In [ ]:
new_data.to_csv('new_data_wiki.csv', index = False, header=True)

###### 

------------------

### Insertion des données

- requête sql

In [ ]:
# connexion à la bdd collection
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection4"
)
cur = conn.cursor()

In [ ]:
sql1 = """SELECT origine.nom
          FROM origine;
"""
sql2 = """SELECT producteur.nom
          FROM producteur;
"""
sql3 = """SELECT produit.nom
          FROM produit;
"""

In [ ]:
# on vérifie que l'origine existe déjà
for pays in new_data['origine'].values:
    print(pays)
    cur.execute("SELECT origine.nom FROM origine WHERE origine.nom = (%s) ", (pays,))
    verif = cur.fetchone()
    if not verif:
        cur.execute("INSERT INTO origine (nom) VALUES(%s)", (pays,))
conn.commit()

In [ ]:
# # on vérifie que le producteur existe déjà
# for producteur in new_data['producteur'].values:
#     cur.execute("SELECT producteur.nom FROM producteur WHERE producteur.nom = (%s) ", (producteur,))
#     verif = cur.fetchone()
#     if not verif:
#         cur.execute("INSERT INTO producteur (nom) VALUES(%s)", (producteur,))
# conn.commit()

In [ ]:
data_co = new_data[['producteur', 'origine']]
# data_co = data_co.reset_index(drop=True, inplace=True)
print(data_co)

In [ ]:
liste_co = []
with open('new_data_wiki.csv', newline='', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        if row[1:3] not in liste_co:
            liste_co.append((row[1:3]))
liste_co = liste_co[1:]
print(liste_co[0:10])

In [ ]:
for producteur, pays in liste_co:
#     print(producteur)
#     print(pays)
    cur.execute(f"""SELECT origine.code_origine 
        FROM origine
        WHERE origine.nom = '{pays}';
        """)
    
    origine = cur.fetchone()[0]
    #print(origine)
    #pays = str(pro[0])
    #print(pays)

    cur.execute(f"INSERT INTO producteur (nom, code_origine) VALUES ('{producteur}', {origine})")

cur.close()
conn.commit()
conn.close()

In [ ]:
# insertion fichuer csv
# prod
# je crée une liste regroupant les pays et les producteurs
liste_groupe = []
with open('new_data_wiki.csv', newline='', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        liste_groupe.append((row[0:2]))
liste_groupe = liste_groupe[1:]
print(liste_groupe[0:10])

In [ ]:
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection4"
)
cur = conn.cursor()
for nom, marque in liste_groupe:
    #print(producteur)
    #print(pays)
    cur.execute(f"""SELECT producteur.code_producteur 
        FROM producteur
        WHERE producteur.nom = '{marque}';
        """)
    code = cur.fetchone()[0]
    
    cur.execute(f"INSERT INTO produit (nom, code_producteur) VALUES ('{nom}', {code})")
print(type(cur))   
cur.close()
conn.commit()
conn.close()

###### 

- traduction nom pays

In [ ]:
# pip install translate

In [17]:
from translate import Translator

In [18]:
translator= Translator(to_lang="fr")

In [19]:
# translation = translator.translate("United Kingdom")

In [20]:
# translation

In [21]:
import pandas as pd

csv_file = 'rum_data.csv'
liste_rhum = pd.read_csv(csv_file)

liste_rhum.head(5)

,name,company,country,price,ratings,score,type,rum_url,img_url,br_score
0,10000 Drops Silver,1000Drops,United States,0,0,0.0,Light,/brands/10972-10000-drops-silver,https://d1jtwiy8m5zi8a.cloudfront.net/assets/d...,4.775363
1,10000 Drops Spiced,1000Drops,United States,0,1,4.0,Spiced,/brands/7354-10000-drops-spiced,https://d19vk5i0q1xlsm.cloudfront.net/uploads/...,4.711955
2,1000 Hills Gold,1000 Hills Distillery,Rwanda,0,1,4.0,Light,/brands/5285-1000-hills-gold,https://d19vk5i0q1xlsm.cloudfront.net/uploads/...,4.711955
3,100 Fuegos Buckeye 2-Year,100 Fuegos,Ecuador,0,0,0.0,Gold,/brands/9037-100-fuegos-buckeye-2-year,https://d1jtwiy8m5zi8a.cloudfront.net/assets/d...,4.775363
4,100 Fuegos 8-Year,101 Fuegos,Ecuador,0,0,0.0,Aged,/brands/1834-100-fuegos-8-year,https://d1jtwiy8m5zi8a.cloudfront.net/assets/d...,4.775363


In [22]:
dico = {}
liste_car = ['Ã¡', 'Ã»', 'Ã¢', 'Ã©', 'â€™', 'Â°', 'Ã¨', 'Ã³', 'Ã±', '\'', 'Ã£','Ã§' ]
liste_remp = ['a', 'u', 'a', 'e', '', '°', 'e', 'o', 'n', '', 'a', 'c']
# création du dictionnaire avec une compréhension de dictionnaire
dico = {k: v for k, v in zip(liste_car, liste_remp)}

In [23]:
liste_rhum.replace({'country': dico}, regex=True,inplace=True)


In [24]:
liste_pays = []
for pays in liste_rhum['country'].values:
    if pays not in liste_pays:
            liste_pays.append(pays)
liste_pays = liste_pays[1:]
print(liste_pays)

['Rwanda', 'Ecuador', 'Trinidad and Tobago', 'Panama', 'Switzerland', 'Netherlands', 'Germany', 'Belgium', 'Saint Lucia', 'Belize', 'India', 'Australia', 'Brazil', 'Barbados', 'Portugal', 'Virgin Islands, U.S.', 'Martinique', 'Mexico', 'United Kingdom', 'Colombia', 'Spain', 'Denmark', 'Caribbean', 'Cuba', 'Dominican Republic', 'Guyana', 'Guatemala', 'Jamaica', 'Mauritius', 'Guadeloupe', 'British Virgin Islands', 'Puerto Rico', 'Croatia', 'Bahamas', 'Sri Lanka', 'Finland', 'Turks and Caicos Islands', 'Antigua and Barbuda', 'Reunion', 'Lithuania', 'Haiti', 'Sweden', 'Grenada', 'Venezuela', 'Fiji', 'Dominica', 'Vietnam', 'Saint Kitts And Nevis', 'Bermuda', 'Cayman Islands', 'Suriname', 'New Zealand', 'Thailand', 'Estonia', 'United States Minor Outlying Islands', 'Italy', 'France', 'Czech Republic', 'Indonesia', 'Peru', 'Canada', 'Costa Rica', 'USA', 'Nicaragua', 'Bulgaria', 'Romania', 'Venezuela, Bolivarian Republic of', 'Hungary', 'El Salvador', 'Trinidad', 'Greece', 'Philippines', 'Sout

In [25]:
pays_traduit = []
liste_pays1 = liste_pays[:60]
print(len(liste_pays1))

liste_pays2 = liste_pays[60:]
print(len(liste_pays2))
for pays in liste_pays1:
    translation = translator.translate(f"{pays}")
#     print(translation)
    pays_traduit.append(translation)
print(pays_traduit)

60
49
['Rwanda', 'Équateur', 'Trinité-et-Tobago', 'Panama', 'Suisse', 'Netherlands', 'Allemagne', 'Belgique', 'Sainte-Lucie', 'Belize', 'Inde', 'Australie', 'Brésil', 'Barbade', 'ortugal', 'Îles Vierges (États-Unis)', 'Martinique', 'Mexique', 'Royaume-Uni', 'Colombie', 'Spain', 'Danemark', 'Caraïbes', 'Cuba', 'République Dominicaine', 'Guyane', 'Guatemala', 'Jamaïque', 'Maurice', 'Guadeloupe', 'Îles Vierges britanniques', 'Puerto Rico', 'Croatie', 'Bahamas', 'Sri Lanka', 'en Finlande', 'Îles Turques et Caïques', 'Antigua-et-Barbuda', 'La Réunion', 'Lituanie', 'Haïti', 'Suède', 'Grenade', 'Venezuela', 'Fidji', 'Dominique', 'Vietnam', 'Saint-Kitts-et-Nevis', 'Bermudes', 'Cayman Islands', 'Suriname', 'Nouvelle Zélande', 'Thaïlande', 'Estonie', 'Îles mineures éloignées des États-Unis', 'Italie', 'France', 'République Tchèque', 'Indonésie', 'Pérou']


In [27]:
f = pd.DataFrame(pays_traduit)
df.to_csv('pays_traduit1.csv')

In [11]:
pays_traduit2= []
liste_pays2 = liste_pays[60:]
print(len(liste_pays2))
for pays in liste_pays2:
    translation = translator.translate(f"{pays}")
#     print(translation)
    pays_traduit2.append(translation)
print(pays_traduit2)

49
['Canada', 'Costa Rica', 'États-Unis', 'Nicaragua', 'Bulgarie', 'Roumanie', 'Venezuela', 'Hongrie', 'Salvador', 'Trinidad', 'Grèce', 'Philippines', 'Afrique du Sud', 'Curaçao', 'Madagascar', 'Uruguay', 'Autriche', 'République populaire de Taiwan', 'Paraguay', 'Cambodge', 'Slovaquie', 'Polynésie Française', 'Japon', 'Argentine', 'Saint-Martin', 'Kenya', 'Népal', 'Malte', 'Seychelles', 'Myanmar', 'Émirats Arabes Unis', 'Aruba', 'Honduras', 'Mexique', 'Pologne', 'Anguilla', 'Inconnu', 'Bonaire, Saint-Eustache et Saba', 'Saint-Barthélemy', 'Andorre', 'Antilles Néerlandaises', 'Jamaïque et Guyane', 'Sint Maarten', 'SAINT-VINCENT-ET-LES-', 'Liechtenstein', 'Antilles anglaises', 'Cap Vert', 'Brasil', 'Irlande']


In [12]:
liste_pays_traduit = zip(pays_traduit, pays_traduit)

NameError: name 'pays_traduit' is not defined

In [13]:
import pandas as pd

In [14]:
df = pd.DataFrame(pays_traduit2)

In [15]:
df.to_csv('pays_traduit2.csv')

###### 

- Test update table utilisateur et collection

In [ ]:
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection4"
)
cur = conn.cursor()